<a href="https://colab.research.google.com/github/laure-m/UCLA_AUD_IDEAS_TechQ1/blob/main/AUD_ESRGAN_upRESer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUD.IDEAS.TECHNOLOGY.STUDIO upRES-er NOTEBOOK
Use this colab to increase the resolution of images ran in VQGANCLIP or StyleTransfer from Studio     

This coLab is based off of the original "ESRGAN (new arch.) on Colab" model found here:  https://github.com/xinntao/ESRGAN and curated by M. Ahabb (Ahabbscience Studio)     

**IMPORTANT:** Read all text before running the cell as some directions can change parameters or might download to the incorrect folder 
Read the output of each cell after you run it > if you get an error or something didnt load or it cant find a folder or image then SOMETHING IS WRONG.

#**PART 01 | THE SETUP**

In [ ]:
#@title **STEP 00:**  Mount to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title **STEP 01:** Change Directory + Download ESRGAN
#@markdown you need to first create a folder named "upRes-main" in your Google Drive    

#@markdown If you have already cloned this Repo then you will get a fatal notice saying it cannot download because it already exists.
%cd //content/drive/MyDrive/upRes-main
!git clone https://github.com/xinntao/ESRGAN

#**PART 02 | UPLOAD IMAGES + DOWNLOAD MODELS** 

In [ ]:
#@title **STEP 02:** Upload images from your computer
#@markdown Remember to delete any old images in the LR folder before this step
%cd /content/drive/MyDrive/upRes-main/ESRGAN/LR

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

In [ ]:
#@title **STEP 03:** Download pretrained upResing models
%cd /content/drive/MyDrive/upRes-main/ESRGAN
import gdown
print("Downloading pretrained models")
output1 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/PPON_D.pth'
output4 = '/content/drive/MyDrive/upRes-main/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True) 
print ('Downloading RRDB_PSNR_x4.pth')
gdown.download('https://drive.google.com/uc?id=1pJ_T-V1dpb1ewoEra1TGSWl5e6H7M4NN', output2, quiet=True)
print ('Downloading PPON_D.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=1Fr5aKCD6mw6P-hI0BZr6My2gHNhtUk-V', output3, quiet=True) 
print ('Downloading PPON_G.pth by Zheng Hui')
gdown.download('https://drive.google.com/uc?id=12uR3BSftNA0HDYiKda23GyAj_crpSjOm', output4, quiet=True)  

#**PART 03 | CHOOSE A MODEL AND UPRES IT** 

In [ ]:
#@title **STEP 04:** Upscale the images / frames 
%cd /content/drive/MyDrive/upRes-main/ESRGAN

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_ESRGAN_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth']  
device = torch.device(Choose_device) 


test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}.png'.format(base), output)

In [ ]:
#@title **STEP 05:** Make a ZIP file of the results

!zip -r results.zip /content/drive/MyDrive/upRes-main/ESRGAN/results

In [ ]:
#@title **STEP 06:** Download the ZIP file
#@markdown You can also just manually download the results folder
files.download('results.zip')

This notebook was put together by Laure Michelon at https://github.com/laure-m